In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
from torch.utils.data import DataLoader

In [8]:
eng_train = pd.read_csv("./iwslt-vi-en/train.tok.en",sep="\t",header=None)
viet_train = pd.read_csv("./iwslt-vi-en/train.tok.vi",sep="\t",header=None)

In [9]:
eng_val = pd.read_csv("./iwslt-vi-en/dev.tok.en",sep="\t",header=None)
viet_val = pd.read_csv("./iwslt-vi-en/dev.tok.vi",sep="\t",header=None)

In [10]:
eng_test = pd.read_csv("./iwslt-vi-en/test.tok.en",sep="\t",header=None)
viet_test = pd.read_csv("./iwslt-vi-en/test.tok.vi",sep="\t",header=None)

In [18]:
def split(df):
    df['tokenized'] = df[0].apply(lambda x:x.split( ))
    return df

In [55]:
eng_train = split(eng_train)
viet_train = split(viet_train)

eng_test = split(eng_test)
viet_test = split(viet_test)

eng_val = split(eng_val)
viet_val = split(viet_val)

In [21]:
print(eng_train.shape)
eng_train.head()

(133168, 2)


,0,tokenized
0,Rachel Pike : The science behind a climate hea...,"[Rachel, Pike, :, The, science, behind, a, cli..."
1,"In 4 minutes , atmospheric chemist Rachel Pike...","[In, 4, minutes, ,, atmospheric, chemist, Rach..."
2,I &apos;d like to talk to you today about the ...,"[I, &apos;d, like, to, talk, to, you, today, a..."
3,Headlines that look like this when they have t...,"[Headlines, that, look, like, this, when, they..."
4,They are both two branches of the same field o...,"[They, are, both, two, branches, of, the, same..."


In [5]:
print(viet_train.shape)
viet_train.head()

(132449, 1)


,0
0,Khoa_học đằng_sau một tiêu_đề về khí_hậu
1,"Trong 4 phút , chuyên_gia hoá_học khí_quyển Ra..."
2,Tôi muốn cho các bạn biết về sự to_lớn của nhữ...
3,Có những dòng trông như thế_này khi bàn về biế...
4,Cả hai đều là một nhánh của cùng một lĩnh_vực ...


In [6]:
eng.iloc[1][0]

'In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .'

In [24]:
next(iter(dataloader['train']))

[[('And', 'So', 'So', 'And', 'For', 'Trophic', 'When', 'I', 'The', 'Well'),
  ('you',
   'what',
   'this',
   'then',
   'every',
   'cascades',
   'you',
   'think',
   'box',
   ','),
  ('can', 'if', 'is', 'of', '100', 'tell', 'ask', 'it', 'jellyfish', 'less'),
  ('see', 'we', 'what', 'course', 'girls', 'us', ',', 'comes', ',', 'than'),
  ('that', 'could', 'I', 'I', 'with', 'that', '&quot;', 'when', 'the', 'one'),
  ('the',
   'put',
   'mean',
   'have',
   'an',
   'the',
   'Where',
   'we',
   'deadliest',
   'comes'),
  ('signature',
   'all',
   'with',
   'to',
   'emotional',
   'natural',
   'are',
   'start',
   'venom',
   'from'),
  ('of',
   'that',
   'a',
   'milk',
   'disturbance',
   'world',
   'the',
   'to',
   'in',
   'the'),
  ('a',
   'together',
   'failing',
   'the',
   'diagnosed',
   'is',
   'Indian',
   'value',
   'all',
   'United'),
  ('stream',
   'to',
   'governance',
   'goats',
   ',',
   'even',
   'Googles',
   'the',
   'of',
   'States'),


In [63]:
print("train",eng_train.shape,viet_train.shape)
print("val",eng_val.shape,viet_val.shape)
print("test",eng_test.shape,viet_test.shape)

train (133168, 3) (132449, 3)
val (1268, 3) (1266, 3)
test (1553, 3) (1546, 3)


In [34]:
all_en_toks = []
for i,row in eng_train.iterrows():
    all_en_toks+=row['tokenized']

In [35]:
all_en_toks

['Rachel',
 'Pike',
 ':',
 'The',
 'science',
 'behind',
 'a',
 'climate',
 'headline',
 'In',
 '4',
 'minutes',
 ',',
 'atmospheric',
 'chemist',
 'Rachel',
 'Pike',
 'provides',
 'a',
 'glimpse',
 'of',
 'the',
 'massive',
 'scientific',
 'effort',
 'behind',
 'the',
 'bold',
 'headlines',
 'on',
 'climate',
 'change',
 ',',
 'with',
 'her',
 'team',
 '--',
 'one',
 'of',
 'thousands',
 'who',
 'contributed',
 '--',
 'taking',
 'a',
 'risky',
 'flight',
 'over',
 'the',
 'rainforest',
 'in',
 'pursuit',
 'of',
 'data',
 'on',
 'a',
 'key',
 'molecule',
 '.',
 'I',
 '&apos;d',
 'like',
 'to',
 'talk',
 'to',
 'you',
 'today',
 'about',
 'the',
 'scale',
 'of',
 'the',
 'scientific',
 'effort',
 'that',
 'goes',
 'into',
 'making',
 'the',
 'headlines',
 'you',
 'see',
 'in',
 'the',
 'paper',
 '.',
 'Headlines',
 'that',
 'look',
 'like',
 'this',
 'when',
 'they',
 'have',
 'to',
 'do',
 'with',
 'climate',
 'change',
 ',',
 'and',
 'headlines',
 'that',
 'look',
 'like',
 'this',
 '

In [36]:
def get_all_toks(df):
    all_toks = []
    for i,row in df.iterrows():
        all_toks+=row['tokenized']
    return all_toks

In [39]:
all_en_toks = get_all_toks(eng_train)
all_viet_toks = get_all_toks(viet_train)

In [38]:
all_viet_toks

['Rachel',
 'Pike',
 ':',
 'The',
 'science',
 'behind',
 'a',
 'climate',
 'headline',
 'In',
 '4',
 'minutes',
 ',',
 'atmospheric',
 'chemist',
 'Rachel',
 'Pike',
 'provides',
 'a',
 'glimpse',
 'of',
 'the',
 'massive',
 'scientific',
 'effort',
 'behind',
 'the',
 'bold',
 'headlines',
 'on',
 'climate',
 'change',
 ',',
 'with',
 'her',
 'team',
 '--',
 'one',
 'of',
 'thousands',
 'who',
 'contributed',
 '--',
 'taking',
 'a',
 'risky',
 'flight',
 'over',
 'the',
 'rainforest',
 'in',
 'pursuit',
 'of',
 'data',
 'on',
 'a',
 'key',
 'molecule',
 '.',
 'I',
 '&apos;d',
 'like',
 'to',
 'talk',
 'to',
 'you',
 'today',
 'about',
 'the',
 'scale',
 'of',
 'the',
 'scientific',
 'effort',
 'that',
 'goes',
 'into',
 'making',
 'the',
 'headlines',
 'you',
 'see',
 'in',
 'the',
 'paper',
 '.',
 'Headlines',
 'that',
 'look',
 'like',
 'this',
 'when',
 'they',
 'have',
 'to',
 'do',
 'with',
 'climate',
 'change',
 ',',
 'and',
 'headlines',
 'that',
 'look',
 'like',
 'this',
 '

In [41]:
from collections import Counter

PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(len(all_tokens)))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token



In [44]:
len(en_id2token)

54171

In [45]:
viet_token2id, viet_id2token = build_vocab(all_viet_toks)

In [46]:
len(viet_token2id)

42150

In [51]:
def token2index_dataset(df,token2id):
    indices_data = []
    for tokens in df['tokenized']:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    df['idized'] = indices_data
    return df

In [54]:
viet_val.head()

,0
0,"Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ..."
1,Tôi đã rất tự_hào về đất_nước tôi .
2,"Ở trường , chúng_tôi dành rất nhiều thời_gian ..."
3,Mặc_dù tôi đã từng tự_hỏi không biết thế_giới ...
4,"Khi tôi lên 7 , tôi chứng_kiến cảnh người_ta x..."


In [56]:
eng_train = token2index_dataset(eng_train,en_token2id)
eng_val = token2index_dataset(eng_val,en_token2id)
eng_test = token2index_dataset(eng_test,en_token2id)

viet_train = token2index_dataset(viet_train,viet_token2id)
viet_val = token2index_dataset(viet_val,viet_token2id)
viet_test = token2index_dataset(viet_test,viet_token2id)

In [57]:
eng_train.head()

,0,tokenized,idized
0,Rachel Pike : The science behind a climate hea...,"[Rachel, Pike, :, The, science, behind, a, cli...","[6554, 17190, 51, 54, 336, 587, 8, 740, 5466]"
1,"In 4 minutes , atmospheric chemist Rachel Pike...","[In, 4, minutes, ,, atmospheric, chemist, Rach...","[127, 3007, 460, 2, 9130, 6555, 6554, 17190, 3..."
2,I &apos;d like to talk to you today about the ...,"[I, &apos;d, like, to, talk, to, you, today, a...","[10, 169, 46, 5, 168, 5, 13, 179, 33, 4, 751, ..."
3,Headlines that look like this when they have t...,"[Headlines, that, look, like, this, when, they...","[24418, 9, 123, 46, 18, 65, 24, 23, 5, 39, 25,..."
4,They are both two branches of the same field o...,"[They, are, both, two, branches, of, the, same...","[106, 22, 412, 117, 5068, 6, 4, 155, 719, 6, 9..."


In [58]:
viet_train.head()

,0,tokenized,idized
0,Khoa_học đằng_sau một tiêu_đề về khí_hậu,"[Khoa_học, đằng_sau, một, tiêu_đề, về, khí_hậu]","[2716, 1360, 6, 3493, 30, 881]"
1,"Trong 4 phút , chuyên_gia hoá_học khí_quyển Ra...","[Trong, 4, phút, ,, chuyên_gia, hoá_học, khí_q...","[203, 425, 395, 2, 919, 1094, 2071, 6706, 1689..."
2,Tôi muốn cho các bạn biết về sự to_lớn của nhữ...,"[Tôi, muốn, cho, các, bạn, biết, về, sự, to_lớ...","[35, 70, 26, 16, 11, 59, 30, 37, 990, 9, 8, 10..."
3,Có những dòng trông như thế_này khi bàn về biế...,"[Có, những, dòng, trông, như, thế_này, khi, bà...","[206, 8, 766, 382, 51, 394, 40, 701, 30, 722, ..."
4,Cả hai đều là một nhánh của cùng một lĩnh_vực ...,"[Cả, hai, đều, là, một, nhánh, của, cùng, một,...","[1627, 126, 119, 4, 6, 3723, 9, 159, 6, 664, 1..."


In [60]:
class Vietnamese(Dataset):
    def __init__(self, en_df,v_df):
        self.en_df = en_df
        self.v_df = v_df
        
    def __len__(self):
        return len(self.en_df)

    def __getitem__(self, idx):
        english = self.en_df.iloc[idx]['idized']
        vietnam = self.v_df.iloc[idx]['idized']
        return [english,vietnam]

In [61]:
bs = 10
transformed_dataset = {'train': Vietnamese(eng_train,viet_train),
                       'validate': Vietnamese(eng_val,viet_val),
                       'test': Vietnamese(eng_test,viet_test)
                                               }

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs,
                        shuffle=True, num_workers=0) for x in ['train', 'validate','test']}

In [64]:
next(iter(dataloader['train']))

[[tensor([  137,   162,    17,    54,   162, 10908,    30,    44,  7309,    55]),
  tensor([144,  38, 180, 359,   4,  51,  75,  12,  51, 153]),
  tensor([  24,  897,    2,   53,  937,  678,  225,    2,   10, 1078]),
  tensor([  176,   233,    14,  3385, 10529,     2,    14,    19,    98,    14]),
  tensor([ 29,  11,  15,   3,   5,   9, 133,  72,  31,  68])],
 [tensor([5730, 4679,   97,  672,  922, 3556,   22,  459, 4366,   97]),
  tensor([    4,  6095,     4,    44,    87, 25126,   444,    92,    58,     4]),
  tensor([ 5502, 17667,  9635,    15,     2,   502,    18,   601,    97,   629]),
  tensor([ 406,  906,  143,  102,  108,    8, 2382,  676,    4,  403]),
  tensor([  14, 3112,  128,   53,  232, 1853,   43,    9,    8,   32]),
  tensor([  287,   815, 19659,  3844,    18,    30,    20,    44,  4047,   106]),
  tensor([   3,   81,    5, 1734,   48,  548,  974,    2, 6645,   81])]]